In [82]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import os
from datetime import datetime
import shutil
import re
import pdfplumber
import pandas as pd

In [83]:
download_dir = 'C:/Users/andre/Downloads'
newsletter_folder = 'C:/Jared Dillian'

pdf_index_num = 1  # 1st, 2nd, 3rd PDF etc. on page

In [84]:
creds_df = pd.read_csv('c:/creds/creds.csv')

my_username = creds_df.loc[0, 'username']
my_password = creds_df.loc[0, 'password']

In [85]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [86]:
driver.get('https://mauldineconomics.com')
time.sleep(2.5)
driver.maximize_window()

In [87]:
decline_cookies = driver.find_element(By.ID, "CybotCookiebotDialogBodyButtonDecline")
decline_cookies.click()

In [88]:
account_link = driver.find_element(By.XPATH, "//a[@class='nav-link px-0' and contains(text(), 'My Account')]")
account_link.click()

In [89]:
username = driver.find_element(By.NAME, 'username')
username.send_keys(my_username)
time.sleep(2)
username = driver.find_element(By.NAME, 'password')
username.send_keys(my_password)
time.sleep(2)

In [90]:
login_btn = driver.find_element(By.XPATH, "//button[@type='submit' and contains(@class, 'btn') and contains(@class, 'u-btn-primary') and contains(text(), 'Log In')]")
login_btn.click()
time.sleep(4)

In [91]:
driver.get('https://www.mauldineconomics.com/strategic-portfolio/')
time.sleep(4)

In [92]:
links = driver.find_elements(By.XPATH, "//a[contains(@class, 'u-link-v2')]")
print(f"Found {len(links)} 'u-link-v2' elements.")

Found 10 'u-link-v2' elements.


In [93]:
for l in links:
    print(l.get_attribute('href'))

https://www.mauldineconomics.com/strategic-portfolio/the-life-hedge
https://www.mauldineconomics.com/strategic-portfolio/people-are-bullish-about-stocks
https://www.mauldineconomics.com/strategic-portfolio/the-charts-told-us
https://www.mauldineconomics.com/strategic-portfolio/everything-is-awesome.-not-for-long
https://www.mauldineconomics.com/strategic-portfolio/sell-tlt-and-tua
https://www.mauldineconomics.com/strategic-portfolio/a-respectable-year
https://www.mauldineconomics.com/strategic-portfolio/sell-angl
https://www.mauldineconomics.com/strategic-portfolio/the-battle-with-bridgewater
https://www.mauldineconomics.com/strategic-portfolio/the-war-portfolio
https://www.mauldineconomics.com/strategic-portfolio/bullfighting


In [94]:
pdf_index = pdf_index_num - 1

if links:
    links[pdf_index].click()

In [95]:
def wait_for_download(directory, timeout=10):
    """Wait for the first new file to appear in the directory within the timeout period."""
    files_before = set(os.listdir(directory))
    elapsed_time = 0
    while elapsed_time < timeout:
        time.sleep(1)  # Check every second
        current_files = set(os.listdir(directory))
        new_files = current_files - files_before
        if new_files:
            return new_files.pop()  # Return the first new file detected
        elapsed_time += 1
    raise FileNotFoundError("No new file appeared in the directory within the timeout period.")


In [96]:
try:
    pdf_link = driver.find_element(By.XPATH, "//a[contains(text(), 'Download PDF')]")
    pdf_link.click()
    downloaded_file_name = wait_for_download(download_dir)
except Exception as e:
    print(e)
    print("If you're seeing this it probably means this was an alert rather than a newsletter with a pdf version.")

In [97]:
time.sleep(4)
log_out_link = driver.find_element(By.XPATH, "//a[contains(text(), 'Log Out')]")
log_out_link.click()

In [98]:
time.sleep(4)
driver.quit()

In [99]:
source_file_path = os.path.join(download_dir, downloaded_file_name)
destination_file_path = os.path.join(r"C:\Jared Dillian", downloaded_file_name)

shutil.move(source_file_path, destination_file_path)

'C:\\Jared Dillian\\JDSP_Apr_2024-CC.pdf'

In [100]:
pdf_path = destination_file_path

In [101]:
def extract_information(pdf_path):
    # Lists to store extracted information
    titles = []
    dates = []
    headings = []

    # Open the PDF file
    with pdfplumber.open(pdf_path) as pdf:
        # Iterate through each page
        for page in pdf.pages:
            # Extract text from the page
            text = page.extract_text()

            # Example patterns to search for a date and titles
            date_pattern = r"\b(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{1,2},\s\d{4}\b"
            title_pattern = r"^[A-Z].*$"  # Assumes titles are lines starting with capital letters and are standalone

            # Find all dates and titles using regular expressions
            found_dates = re.findall(date_pattern, text)
            found_titles = re.findall(title_pattern, text, flags=re.MULTILINE)

            # Add found items to respective lists
            dates.extend(found_dates)
            titles.extend(found_titles)

            # Finding headings can be more complex and might need specific patterns based on the document structure
            # For example, if headings are known to be uppercase or have certain keywords:
            for line in text.split('\n'):
                if line.isupper() or 'HEADING_KEYWORD' in line:
                    headings.append(line)

    return titles, dates, headings


In [102]:
titles, dates, headings = extract_information(pdf_path)

In [103]:
print("Title:", titles[3])
print("Date:", dates[0])


Title: The Life Hedge
Date: April 16, 2024


In [104]:
new_file_name = titles[3]
dt = pd.to_datetime(dates[0]).strftime('%Y%m%d')

In [105]:
new_file_name = new_file_name.replace(' ', '_') + '.pdf'
new_file_name = dt + '_' + new_file_name

In [106]:
if os.path.exists(os.path.join(newsletter_folder, new_file_name)):
    os.remove(os.path.join(newsletter_folder, new_file_name))

os.rename(os.path.join(newsletter_folder, downloaded_file_name), os.path.join(newsletter_folder, new_file_name))